In [1]:
print("Preparing environment...")

import platform

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from astropy import table as tbl
from astropy import units as u
from astropy.io import fits
from scipy import constants as c
from scipy import interpolate
from tqdm.auto import tqdm

# %matplotlib widget

print("Done.")

Preparing environment...
Done.


In [2]:
### All functions

# Function to use appropriate OS directory structure
def GetDirStruct():
    match platform.system():
        case "Windows":
            dirPrefix = "D:/"
        case "Linux" | "Ubuntu":
            dirPrefix = "/mnt/d/"
        case "macOS" | "Darwin":
            dirPrefix = "/Volumes/Storage/"
        case _:
            raise Exception(f"OS not recognised: \"{platform.system()}\". Please define a custom switch inside GetDirStruct().")
    return dirPrefix


# Function to grab list of spectra in a directory
def GetSpec():
    dirPrefix = GetDirStruct()
    specDir = dirPrefix + specFolder
    specList = !bash -c "ls {specDir}*1D.fits"
    specList = [file.split('/')[-1] for file in specList]
    specNames = [file.split('_')[0] for file in specList]
    return specDir, specList, specNames


# Function to import files to dictionary
def ImportSpec(specList, specDir, specNames):
    specData = {}
    for file, name in tqdm(zip(specList, specNames), desc="Importing spectra", total=len(specList)):
        specRaw = fits.open(specDir + file)
        specFlux = specRaw[1].data * u.Jy
        specWave = specRaw[9].data * u.m
        specData[name] = tbl.QTable([specWave, specFlux], names=("Wavelength", "Flux"))
        specRaw.close()
    return specData


# Function to plot the spectra
def PlotSpec(specData, specList, specNames):
    !bash -c "mkdir -p ../../Working_Directory/Apo_Phot_Utils/{outputFolder}plots"
    for file, name in tqdm(zip(specList, specNames), desc="Plotting galaxies", total=len(specList)):
        plt.plot(specData[name]["Wavelength"], specData[name]["Flux"])
        plt.xscale('log')
        plt.savefig(f"../../Working_Directory/Apo_Phot_Utils/{outputFolder}plots/{name}.png")
        plt.close()
    return

# Function to grab filter curves
def GetFilter():
    dirPrefix = GetDirStruct()
    filterDir = dirPrefix + filterFolder
    filterList = !bash -c "ls {filterDir}"
    filterList = [file.split('/')[-1] for file in filterList]
    filterNames = [file.split('_')[0] for file in filterList]
    filterData = {}
    for file, name in tqdm(zip(filterList, filterNames), desc="Importing filters", total=len(filterList)):
        filterData[name] = tbl.QTable.read(filterDir + file, format="ascii")
        filterData[name]["Microns"].unit = u.um
        filterData[name].rename_column("Microns", "Wavelength")
    return filterData

# Function to parse tables and set any rows with NaNs to zero, to avoid errors
def RemoveNaNs(tableObj):
    for col in tableObj.colnames:
        for x in range(0, len(tableObj[col]), 1):
            if np.isnan(tableObj[col][x]):
                tableObj[col][x] = 0
    return tableObj

# Function to interpolate datapoints
def InterpFunc(funcXs, funcYs):
    funcCubic = interpolate.interp1d(funcXs, funcYs, kind='cubic')
    return funcCubic

# Function to find grid overlap for convolutions of target using input
def FindGrid(targetGrid, inputGrid, inputData):
    # Sort all arrays
    idxSorted = np.argsort(inputGrid)
    sortedGrid = inputGrid[idxSorted]
    sortedTarget = np.sort(targetGrid)
    # Find first value in input that overlaps with target
    if sortedGrid[0] < sortedTarget[0]:
        idxLeft = np.searchsorted(sortedGrid, sortedTarget[0], side="right")
        if inputGrid[idxLeft-1] == sortedTarget[0]:
            idxLeft -= 1
    else:
        idxLeft = 0
    # Find last value in input that overlaps with target
    if sortedGrid[-1] > sortedTarget[-1]:   
        idxRight = np.searchsorted(sortedGrid, sortedTarget[-1], side="left")
        if inputGrid[idxRight] != sortedTarget[-1]:
            idxRight -= 1
    else:
        idxRight = -1
    # Return the section of input that overlaps target
    overlapGrid = sortedGrid[idxLeft:idxRight]
    overlapData = inputData[idxSorted][idxLeft:idxRight]
    return overlapGrid, overlapData

# Function to manage convolution calculation from first array onto second array
def ConvolveFunc(firstXs, firstYs, secondXs, secondYs):
    # Interpolate datapoints of first array
    firstInterp = InterpFunc(firstXs.to(u.m), firstYs)
    # Find relevant convolution grid for first array based on second array
    secondOverlapGrid, secondOverlapData = FindGrid(firstXs, secondXs, secondYs)
    # Convolve first array onto second array's grid
    firstConvolved = firstInterp(secondOverlapGrid.to(u.m))
    return firstConvolved, secondOverlapGrid, secondOverlapData

# Function to convolve spectra and filters onto same grid
def MergeGrids(specFile, filterFile):
    specWave = specFile["Wavelength"]
    specFlux = specFile["Flux"]
    filterWave = filterFile["Wavelength"]
    filterThrough = filterFile["Throughput"]
    # Find mutual overlaps
    specConvolved, filterOverlapGrid, filterOverlapData = ConvolveFunc(specWave, specFlux, filterWave, filterThrough)
    filterConvolved, specOverlapGrid, specOverlapData = ConvolveFunc(filterWave, filterThrough, specWave, specFlux)
    # Save as tables
    specTable = tbl.Table([np.append(specOverlapGrid, filterOverlapGrid).to(u.m), np.append(specOverlapData, specConvolved*u.Jy)], names=("Wavelength", "Flux"))
    specTable.sort("Wavelength")
    filterTable = tbl.Table([np.append(filterOverlapGrid, specOverlapGrid).to(u.m), np.append(filterOverlapData, filterConvolved)], names=("Wavelength", "Throughput"))
    filterTable.sort("Wavelength")
    # Join tables
    mergedTable = tbl.join(specTable, filterTable, keys="Wavelength")
    return mergedTable

# Function to shift flux to photon space
def ShiftPhotonSpace(mergedTable):
    mergedTable["Flux"] *= mergedTable["Wavelength"]
    return mergedTable

# Function to integrate two functions on the same grid
def IntegFunc(firstYs, secondYs, commonGrid):
    funcIntegrated = np.trapz(firstYs * secondYs, x=commonGrid)
    return funcIntegrated

# Function to normalise the throughput to the correct zero-point
def NormaliseValue(specTable, specValue):
    specThrough = specTable["Throughput"]
    specGrid = specTable["Wavelength"]
    # norm_ref = 10**(48.6/(-2.5)) # reference zero magnitude
    normRef = 1 * u.Jy.to(u.W / ((u.m)**2 * u.Hz)) * 10**(-6) # reference flat-value in f_nu
    normYs = normRef * c.c / specGrid**2 * specGrid # not squared, f_lambda in photon_space
    normValue = IntegFunc(specThrough, normYs, specGrid)
    specNormed = specValue / normValue
    return specNormed

# Function to calculate throughput on a merged grid
def CalcThroughput(mergedTable):
    mergedWave = mergedTable["Wavelength"]
    mergedFlux = mergedTable["Flux"]
    mergedThrough = mergedTable["Throughput"]
    mergedOut = IntegFunc(mergedFlux, mergedThrough, mergedWave)
    return mergedOut

# Function to find throughput given a spectrum and filter
def FindThroughput(specFile, filterFile):
    # Convolve onto the same grid
    mergedTable = MergeGrids(specFile, filterFile)
    # Shift to photon space
    mergedTable = ShiftPhotonSpace(mergedTable)
    # Integrate through the filter
    return CalcThroughput(mergedTable), mergedTable

def BalmerBreak(specFile, specName, redshiftFile):
    # Define regions
    balmerLeftRange = np.array([3500, 3650]) * u.angstrom
    balmerRightRange = np.array([3800, 3950]) * u.angstrom
    # Grab relevant grids
    for row in range(0, len(redshiftFile), 1):
        if int(specName) == int(redshiftFile[row][redshiftID]) and redshiftFile[row][redshiftZ] > 0:
            redshiftFactor = 1 + redshiftFile[row][redshiftZ]
            break
        else:
            redshiftFactor = np.nan
    if redshiftFactor != 0:
        balmerLeftWave, balmerLeftSpec = FindGrid(balmerLeftRange, specFile["Wavelength"]/redshiftFactor, specFile["Flux"]*redshiftFactor)
        balmerRightWave, balmerRightSpec = FindGrid(balmerRightRange, specFile["Wavelength"]/redshiftFactor, specFile["Flux"]*redshiftFactor)
        # Shift to photon space
        balmerLeftSpec *= balmerLeftWave
        balmerRightSpec *= balmerRightWave
        # Calculate average flux
        balmerLeftVal = np.sum(balmerLeftSpec) / len(balmerLeftSpec)
        balmerRightVal = np.sum(balmerRightSpec) / len(balmerRightSpec)
        # Calculate ratio
        balmerRatio = balmerRightVal/balmerLeftVal
        balmerVals = [balmerLeftVal, balmerRightVal, balmerRatio]
    return balmerVals

# Function to loop through filters and balmer breaks for one spectrum
def HandleSpectrum(specFile, specName, filterData, redshiftFile):
    specValues = []
    for filter in filterData.values():
        specValue, specTable = FindThroughput(specFile, filter)
        specValues += [NormaliseValue(specTable, specValue)]
    specValues += BalmerBreak(specFile, specName, redshiftFile)
    return specValues

# Function to loop through each spectrum and save values to a table
def LoopSpectra(specData, filterData, redshiftFile):
    !bash -c "mkdir -p ../../Working_Directory/Apo_Phot_Utils/{outputFolder}"
    rows = []
    for specName, specFull in tqdm(zip(specData.keys(), specData.values()), desc="Galaxies", total=len(specData.keys())):
        rows += [[int(specName)] + HandleSpectrum(specFull, specName, filterData, redshiftFile)]
    names = ["ID"] + list(filterData.keys()) + ["Balmer_left", "Balmer_right", "Balmer_ratio"]
    outTable = tbl.Table(rows=rows, names=names)
    outTable.write(f"../../Working_Directory/Apo_Phot_Utils/{outputFolder}throughputs.fits")
    return

In [3]:
print("Importing data...")

# Directories
try:
    filterFolder
    specFolder
    outputFolder
    comparisonFile
except NameError:
    filterFolder = "Throughputs/nircam_throughputs/mean_throughputs/"
    specFolder = "Spectra/HST_Medium/prism_clear_v1.5/prism_clear/"
    outputFolder = "HST_Medium/prism_clear_v1.5/"

# Spectra
try:
    specData
except NameError:
    specDir, specList, specNames = GetSpec()
    specData = ImportSpec(specList, specDir, specNames)

# Filters
try:
    filterData
except NameError:
    filterData = GetFilter()

# Redshift catalogue
try:
    redshiftFile
except NameError:
    redshiftFolder = GetDirStruct() + "Redshifts/Medium_HST.csv"
    redshiftID = "ID"
    redshiftZ = "Assigned_redshift"
    redshiftFile = tbl.Table.read(redshiftFolder)

print("Cleaning data...")

for spectrum in tqdm(specData.values(), desc="Cleaning spectra"):
    spectrum = RemoveNaNs(spectrum)
for filterFile in tqdm(filterData.values(), desc="Cleaning filters"):
    filterFile = RemoveNaNs(filterFile)

print("Plotting images...")

PlotSpec(specData, specList, specNames)

print("Calculating throughputs...")

LoopSpectra(specData, filterData, redshiftFile)

print("Done.")

Importing data...


Importing spectra:   0%|          | 0/577 [00:00<?, ?it/s]

Importing filters:   0%|          | 0/30 [00:00<?, ?it/s]

Cleaning data...


Cleaning spectra:   0%|          | 0/577 [00:00<?, ?it/s]

...

Plotting images...


Plotting galaxies:   0%|          | 0/577 [00:00<?, ?it/s]

Calculating throughputs...


Galaxies:   0%|          | 0/577 [00:00<?, ?it/s]

Done.


In [ ]:
plt.close("all")

# plt.plot(Spectrum["Wavelength"], Spectrum["Flux"])
# plt.plot(Filter["Wavelength"], Filter["Throughput"] * 10 **(-13))
plt.plot(Filter_grid, Filter_conv * 10 ** (-13), color="green")
plt.plot(Filter_grid, Spectrum_flux, color="blue")
plt.plot(Filter_grid, Filter_conv * Spectrum_flux, color="red")